In [1]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('admission_data.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(21)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [3]:
print(features.head())
print (features.shape)

          gre       gpa  rank_1  rank_2  rank_3  rank_4
106  0.972155  0.446965       1       0       0       0
9    0.972155  1.392922       0       1       0       0
61  -0.239793 -0.183673       0       0       0       1
224  1.837832 -1.287291       0       1       0       0
37  -0.586063 -1.287291       0       0       1       0
(360, 6)


In [4]:
np.random.seed(21)

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


In [14]:
# Hyperparameters
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005


In [6]:
n_records, n_features = features.shape
last_loss = None
# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

In [10]:
weights_hidden_output

array([ 0.65768472, -0.28137626])

In [17]:
for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        # TODO: Calculate the output
        hidden_input = np.dot(x,weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_output,weights_hidden_output))

        ## Backward pass ##
        # TODO: Calculate the error
        error = y-output

        # TODO: Calculate error gradient in output unit
        output_error = error*output*(1-output)

        # TODO: propagate errors to hidden layer
        hidden_error = np.dot(output_error,weights_hidden_output)*hidden_output*(1-hidden_output)
            

        # TODO: Update the change in weights
        del_w_hidden_output += output_error * hidden_output
        del_w_input_hidden += hidden_error * x[:, None]

    # TODO: Update weights
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        print (e)
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss



0
Train loss:  0.23025255275405246
90
Train loss:  0.22960072284542718
180
Train loss:  0.22897309660726808
270
Train loss:  0.2283688749295133
360
Train loss:  0.22778728181289093
450
Train loss:  0.22722756402289856
540
Train loss:  0.22668899071621654
630
Train loss:  0.22617085304349513
720
Train loss:  0.22567246373207567
810
Train loss:  0.22519315665195327


In [18]:
# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))


Prediction accuracy: 0.700
